In [9]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [10]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [11]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [12]:
SYSTEM_PROMPT = """
Based on the provided use case specification, please generate a set of test scenarios to thoroughly test the functionality described in the use case.
Rules to predict scenarios:
- Stay close to the details described in the flow.
- Focus on important and likely scenarios, important scenario is the scenario that users are more likely to encounter it. 
- Minimize the appearance of rare scenarios. 
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

In [13]:
usecase_path = r"C:\Users\congc\Desktop\GPT\Dissertation-GPT\dataset\SpecificationData\Shopping System"
save_path = r"C:\Users\congc\Desktop\GPT\Dissertation-GPT\evaluate\prove\Shopping System\ts"

In [14]:
usecase_name_list = []
project_name = os.path.basename(usecase_path)

In [15]:
for filename in os.listdir(usecase_path):
    usecase_name_list.append (filename.split(".txt")[0])
print(usecase_name_list)

['add to cart', 'checkout', 'product information display', 'purchase', 'search']


In [16]:
for usecase_name in usecase_name_list:
    print(usecase_name)
    usecase_directlink = os.path.join(usecase_path,usecase_name+".txt")
    prompt_all = read_file_content(usecase_directlink)
    promptSYSTEM_PROMPT = [
        { "role": "system", "content": SYSTEM_PROMPT},
        { "role": "user", "content": prompt_all}
    ]
    gpt_response = ask(promptSYSTEM_PROMPT, client, model)    
    print("gpt_response \n" +gpt_response)
    write_string_to_file(os.path.join(save_path,f"{usecase_name}.txt"), gpt_response)

add to cart
gpt_response 
1. Add Single Option Product to Cart
2. Add Multi-Option Product to Cart After Selecting an Option
3. Adjust Quantity Before Adding Product to Cart
4. Prevent Adding Product Without Selecting Option
5. Prevent Adding Product Over Maximum Stock
6. Prevent Reducing Quantity Below One
7. Prevent Adding Out of Stock Product to Cart
String has been written to C:\Users\congc\Desktop\GPT\Dissertation-GPT\evaluate\prove\Shopping System\ts\add to cart.txt
checkout
gpt_response 
1. Successful Checkout with Default Options
2. Checkout with Changed Shipping Address
3. Checkout with New Shipping Address
4. Checkout with Changed Payment Method
5. Checkout with Application Voucher Applied
6. Checkout with Shop Voucher Applied
7. Checkout with Changed Shipping Option
8. Checkout with Note for Shop Added
9. Checkout with No Default Shipping Address
10. Checkout with Incorrect Voucher Code
11. Checkout with Disabled Voucher
12. Checkout with Invalid Shipping Address Input
13. C